## Background:
One overarching goal of this research project is to determine the feasibility of using the online Yelp platform to collect business listings for retailers of Electronic Nicotine Delivery Systems (ENDS; e.g., vape shops) in Pennsylvania. To this end, I utilized the Yelp API to collect data for all metropolitan and micropolitan census regions in Pennsylvania (which includes some surrounding states). Each API call was set to maximum search radius of 25 miles, targeting a central zipcode within each identified census region (some geographic overlap was present). 

Data have been collected from the Yelp API at roughly 1-month intervals from September 2016 through present. Data are currently stored in CSV files. Monthly API calls were repeated using several distinct search terms such as "vape", "vaping", and "ecig" to maximize sensitivity. 

As a result of overlap in search radii and search terms, each monthly data file contains many redundant listings. As the searches were repeated several times, search results are also redundant among the data files. Also, several results are outside of Pennsylvania, so additional data cleaning is needed to limit the scope of results to Pennsylvania ENDS retailers. 

Data were collected as part of an ongoing research project at the University of Pittsburgh's [Center for Research on Media, Technology, and Health](http://mth.pitt.edu/) and are stored in [CRMTH's GitHub "YelpEpi" repository](https://github.com/CRMTH/YelpEpi/). For the purposes of this project, I forked that repo to [my personal GitHub "YelpEpi" repository](https://github.com/colditzjb/YelpEpi/).



### Loading the data

First, I needed to clone the repository to my computer. I did this in BASH command line, but I'm including that step for posterity (this may not work as expected within Jupyter).

In [ ]:
#!git clone http://github.com/colditzjb/YelpEpi/

Depending on your current working directory (e.g., where you started Jupyter), this repo may end up in a different location on your own computer. Change this next line to point to the correct directory:

In [3]:
dir_in = '//home/jason/repos/YelpEpi/'

Okay, now that we're (hopefully) on the same page, let's navigate to the data subdirectory and see what we're working with...

In [4]:
import os
data_dir = dir_in+'data/'
files = sorted(os.listdir(data_dir)) # Use sorted() to list them in ascending order
files

['2016-09-02.csv',
 '2016-09-30.csv',
 '2016-10-31.csv',
 '2016-11-30.csv',
 '2017-01-03.csv',
 '2017-01-31.csv',
 '2017-02-28.csv',
 '2017-04-03.csv',
 '2017-05-01.csv',
 '2017-05-30.csv',
 'README.md']

A couple issues:
* There is a README.md file among the CSV data files, so we'll need to ignore that.
* The data collection dates are in the file names, and we'll need those for later.

Let's clean that up a bit:

In [20]:
# Figuring out the slicing options
f = '2017-05-01.csv'
print('extension is:\t'+f[-4:]) # Use this later to select only CSV files
print('filename is:\t'+f[:-4]) # Use this later for parsing out dates from file names

extension is:	.csv
filename is:	2017-05-01


In [22]:
# Making lists of only CSV files
files_csv = []
for f in files:
    if '.csv' in f[-4:]:
        files_csv.append(f)
files = files_csv 
files

['2016-09-02.csv',
 '2016-09-30.csv',
 '2016-10-31.csv',
 '2016-11-30.csv',
 '2017-01-03.csv',
 '2017-01-31.csv',
 '2017-02-28.csv',
 '2017-04-03.csv',
 '2017-05-01.csv',
 '2017-05-30.csv']

Now we'll read in the first CSV file as a Pandas object and examine it.

In [26]:
import pandas as pd
df = pd.read_csv(data_dir+files[0])
df.head(3) # Only display the first 3 records

,termnum,term,category,radius_miles,loci,location,lat,lng,distance,i,...,is_closed,rating,review_count,name,phone,display_address,url,yelpcats,isWTS,isTobShop
0,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549806,-75.491105,NaN,1,...,False,4.0,4,Get Your Vape On,6104218310,610 State Ave | Emmaus PA 18049,http://www.yelp.com/biz/get-your-vape-on-emmau...,vapeshops |,NaN,NaN
1,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549495,-75.597692,NaN,2,...,False,4.5,2,Vape Flow,4846498347,7150 Hamilton Blvd | Trexlertown PA 18087,http://www.yelp.com/biz/vape-flow-trexlertown-...,vapeshops |,NaN,NaN
2,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.629023,-75.477516,NaN,3,...,False,1.0,1,Blue Monkey Vape,6102310555,250 Lehigh Valley Mall | Whitehall PA 18052,http://www.yelp.com/biz/blue-monkey-vape-white...,vapeshops |,NaN,NaN


Next, let's confirm that all of the files are Pandas-readable...

_Spoiler alert:_ they're not all good!

In [29]:
for f in files:
    try:
        df = pd.read_csv(data_dir+f)
        print(f+' is all good')
    except:
        print(f+' is NOT good')

2016-09-02.csv is all good
2016-09-30.csv is all good
2016-10-31.csv is all good
2016-11-30.csv is all good
2017-01-03.csv is NOT good
2017-01-31.csv is NOT good
2017-02-28.csv is NOT good
2017-04-03.csv is NOT good
2017-05-01.csv is all good
2017-05-30.csv is all good


What's going on with the "NOT good" files?!

Let's iterate to find the error...

In [68]:
for line in open(data_dir+'2017-01-03.csv'):
    pass

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2336: invalid continuation byte

Let's examine the file in question...

In [47]:
lineNum = 0
try:
    for line in open(data_dir+'2017-01-03.csv'):
        lineNum += 1
        pass
except:
    print('Last good line was #'+str(lineNum)+' and had this data:\n')
    print(line)


Last good line was #944 and had this data:

0,vape,,25,36,York-Hanover | PA Metro Area; Pennsylvania (Spring Grove | PA),39.9766769,-76.7686615,5516.60113329,18,djs-westgate-beverage-york,False,4.0,1,DJ's Westgate Beverage,7177641550,1550 Kenneth Rd | York PA 17408,https://www.yelp.com/biz/djs-westgate-beverage-york?adjust_creative=wDwCvDADIHyvJYDHOmNK2g&utm_campaign=yelp_api&utm_medium=api_v2_search&utm_source=wDwCvDADIHyvJYDHOmNK2g,beer_and_wine | tobaccoshops | 



At this point, we know that there is a utf-8 encoding error around line # 944 in the '2017-01-03.csv' file (and also in some subsequent files). Upon reviewing the raw data, there weren't any obviously strange text characters, so we're going to try a different encoding strategy.

After some trial-and-error (and various StackOverflow pages), _"latin-1"_ might be a viable encoding strategy when interpreting text from an API that is international in scope. Let's try that...

In [66]:
for f in files:
    try:
        df = pd.read_csv(data_dir+f, encoding='latin-1')
        print(f+' is all good')
    except:
        print(f+' is NOT good')

2016-09-02.csv is all good
2016-09-30.csv is all good
2016-10-31.csv is all good
2016-11-30.csv is all good
2017-01-03.csv is all good
2017-01-31.csv is all good
2017-02-28.csv is all good
2017-04-03.csv is all good
2017-05-01.csv is all good
2017-05-30.csv is all good


__Success - it's all good!__

### Documenting the data

Now that the data files are programatically "all good", let's double check the `head()` and `info()` functions for a couple dataframes with different circumstances:

* 2016-09-02.csv was well-behaved before encoding
* 2017-01-03.csv was not well-behaved before encoding

If everything looks congruent, we will then document (as per the assignment goals):
* Look at each column, describe the data types, and try to understand what the data means? There doesn't appear to be a data dictionary, so it is perfectly OK to say "I don't know" `¯\_(ツ)_/¯`


In [5]:
import pandas as pd
df = pd.read_csv(data_dir+'2016-09-02.csv', encoding='latin-1')
df.info()
df.head(3) # Only display the first 3 records

<class 'pandas.core.frame.DataFrame'>
Int64Index: 929 entries, 0 to 928
Data columns (total 21 columns):
termnum            929 non-null int64
term               929 non-null object
category           0 non-null float64
radius_miles       929 non-null int64
loci               929 non-null int64
location           929 non-null object
lat                929 non-null float64
lng                929 non-null float64
distance           0 non-null float64
i                  929 non-null int64
yelp_id            929 non-null object
is_closed          929 non-null bool
rating             929 non-null float64
review_count       929 non-null int64
name               929 non-null object
phone              926 non-null float64
display_address    929 non-null object
url                929 non-null object
yelpcats           929 non-null object
isWTS              0 non-null float64
isTobShop          0 non-null float64
dtypes: bool(1), float64(8), int64(5), object(7)
memory usage: 127.9+ KB


,termnum,term,category,radius_miles,loci,location,lat,lng,distance,i,...,is_closed,rating,review_count,name,phone,display_address,url,yelpcats,isWTS,isTobShop
0,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549806,-75.491105,NaN,1,...,False,4.0,4,Get Your Vape On,6104218310,610 State Ave | Emmaus PA 18049,http://www.yelp.com/biz/get-your-vape-on-emmau...,vapeshops |,NaN,NaN
1,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549495,-75.597692,NaN,2,...,False,4.5,2,Vape Flow,4846498347,7150 Hamilton Blvd | Trexlertown PA 18087,http://www.yelp.com/biz/vape-flow-trexlertown-...,vapeshops |,NaN,NaN
2,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.629023,-75.477516,NaN,3,...,False,1.0,1,Blue Monkey Vape,6102310555,250 Lehigh Valley Mall | Whitehall PA 18052,http://www.yelp.com/biz/blue-monkey-vape-white...,vapeshops |,NaN,NaN


In [6]:
import pandas as pd
df = pd.read_csv(data_dir+'2017-01-03.csv', encoding='latin-1')
df.info()
df.head(3) # Only display the first 3 records

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1649 entries, 0 to 1648
Data columns (total 22 columns):
termnum            1649 non-null int64
term               1649 non-null object
category           0 non-null float64
radius_miles       1649 non-null int64
loci               1649 non-null int64
location           1649 non-null object
lat                1649 non-null float64
lng                1646 non-null float64
distance           1646 non-null float64
i                  1649 non-null int64
yelp_id            1649 non-null object
is_closed          1649 non-null bool
rating             1649 non-null float64
review_count       1649 non-null int64
name               1649 non-null object
phone              1632 non-null float64
display_address    1649 non-null object
url                1649 non-null object
yelpcats           1649 non-null object
isWTS              0 non-null float64
isTobShop          0 non-null float64
Unnamed: 21        0 non-null float64
dtypes: bool(1), float6

,termnum,term,category,radius_miles,loci,location,lat,lng,distance,i,...,rating,review_count,name,phone,display_address,url,yelpcats,isWTS,isTobShop,Unnamed: 21
0,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549110,-75.491142,5456.363136,1,...,4,4,Get Your Vape On,6104218310,610 State Ave | Emmaus PA 18049,https://www.yelp.com/biz/get-your-vape-on-emma...,vapeshops |,NaN,NaN,NaN
1,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.549495,-75.597692,5461.880656,2,...,1,1,Vape Flow,4846810033,7150 Hamilton Blvd | Trexlertown PA 18087,https://www.yelp.com/biz/vape-flow-trexlertown...,vapeshops |,NaN,NaN,NaN
2,0,vape,NaN,25,1,Allentown-Bethlehem-Easton | PA-NJ Metro Area ...,40.628833,-75.477412,5456.567591,3,...,1,1,Blue Monkey Vape,6102310555,250 Lehigh Valley Mall | Whitehall PA 18052,https://www.yelp.com/biz/blue-monkey-vape-whit...,vapeshops |,NaN,NaN,NaN


One obvious difference between these two files is the presence of an additional, blank "_Unnamed: 21_" column in the second file. That can be ignored, as well as some of the other columns that don't yield useful information. Here's the breakdown of the data dictionary:

00 __termnum:__            [ignore] iterable relating to the search term used in data collection

01 __term:__              the search term used in data collection

02 __category:__           [ignore] empty column

03 __radius_miles:__       [ignore] search radius was 25 miles for all data

04 __loci:__               [ignore] iterable relating to the location searched about

05 __location:__           census tract information for this search

06 __lat: __               latitude of business location

07 __lng:__                longitude of business location

08 __distance:__           [ignore] distance from an arbitrary point to the location

09 __i:__                  [ignore] iterable relating to the search result listing

10 __yelp_id:__            unique Yelp identifier string for this business listing

11 __is_closed:__          is this business marked as closed on Yelp? (True/False)

12 __rating:__             average rating of this business on Yelp (range: 1-5)

13 __review_count:__       number of reviews contributing to the average rating

14 __name:__               business name

15 __phone:__              business phone number

16 __display_address:__    business address

17 __url:__                webpage for this Yelp listing

18 __yelpcats:__           business categories on the Yelp platform 

19 __isWTS:__              empty column intended to track "Vape Shops" from `yelpcats`

20 __isTobShop:__          empty column intended to track "Tobacco Shops" from `yelpcats`

21 __Unnamed: 21:__        [ignore] erroneous column


_Also of note:_ During data collection commas were replaced with "|" bars in string variables so that the resulting CSV files would line up correctly. This may be useful information for splitting string fields later on! 

### Cleaning the data

A few overarching issues for data cleaning:
* Data files are _roughly_ monthly data collection points, but sometimes data were collected at the beginning and end of a particular month. A `timepoint` field might be better than a date field in this circumstance. 
* Some data files have an extra, blank column.
* Some columns aren't useful for the present analyses and can be dropped.
* Redundancy; lots of redundant data!

More nuanced issues for data cleaning:
* Need to extract state abbreviation from the `display_address` field to center-in on Pennsylvania data.
* Need to populate the `isWTS` field to identify businesses marked as "Vape Shops" by Yelp.

Let's do this...


In [36]:
# Create new files with reduced columns and a new column for "timepoint"
import os, csv

files = sorted(os.listdir(data_dir)) # Use sorted() to list them in ascending order

# Reduce the output file to only the variables that we're interested in
def writeLine(timepoint, l):
    v = l.split(',')
    with open(data_dir+'output.csv', 'a+') as output:
        CSVwriter = csv.writer(output, delimiter=',')
        CSVwriter.writerow([timepoint, v[1], v[5], v[6], v[7], v[10], 
                            v[11], v[12], v[13], v[14], v[15], v[16], 
                            v[17], v[18]])

timepoint = 0
for f in files:
    if '.csv' in f[-4:]: # Only include CSV files
        line = 0 # Need to iterate lines to control for header rows
        with open(data_dir+f, encoding='latin-1') as file:
            for l in file:
                if timepoint == 0 and line == 0:
                    writeLine('timepoint',l) # Write header row
                elif line > 0:
                    writeLine(timepoint,l) # Write data
                else:
                    pass
                line = line + 1
            timepoint = timepoint + 1

            


_Note to self:_ Update the remote repository...

In [83]:
#!git add .
#!git commit -m "update files"
#!git config user.email "colditzjb@gmail.com"
#!git push origin master